# Imports

## Useful packages

In [ ]:
import numpy as np
from functools import partial
import pandas as pd
import netCDF4
from scipy.interpolate import interp1d

## plotting

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models.renderers import GlyphRenderer
from bokeh.models import Range1d, LinearAxis
from bokeh.models import HoverTool

# is needed for unlocking bokeh doc
from panel.io import unlocked

pn.extension('katex')
hv.extension('bokeh', width=100)

# pn.config.sizing_mode = 'stretch_both'

## OGGM

In [ ]:
from oggm.core.massbalance import MassBalanceModel, PastMassBalance, ConstantMassBalance
from oggm.utils import ncDataset, date_to_floatyear
from oggm import cfg
from oggm.cfg import SEC_IN_YEAR, SEC_IN_MONTH

cfg.initialize_minimal()

# define some settings

In [ ]:
cfg.PARAMS['use_bias_for_run'] = False

climate_data_dir = 'climate_data/'

# Define Domain

In [ ]:
# glacier top height
max_height = 6000

# glacier bottom height
min_height = 0

# Define default base climates

Values from www.climate-data.org

In [ ]:
base_climate_names = {'Hintereisferner (Mid Latitudes)': '_HEF',
                      'Lewis (Tropics)': '_Lewis'}

# Define Menu

In [ ]:
# define background color of tab menu
menu_background = '#f4f4f4'

# define width for menu-tabs
panel_width = 250

# define width for whole menu
tab_menu_width = 370

# needed for some initialisations for start of app
initial_run = True

## Climate Tab

In [ ]:
# Climate Tab
def set_default_base_climate(event):
    temp_bias_slider.value = 0.
    prcp_fac_slider.value = cfg.PARAMS['prcp_scaling_factor']
    temp_grad_slider.value = cfg.PARAMS['temp_default_gradient'] * 100

base_climate_select = pn.widgets.Select(name='Base Climate',
                                        options=list(base_climate_names.keys()))
base_climate_select.param.watch(set_default_base_climate, ['value'])

y0_slider = pn.widgets.IntSlider(name='Center of Period',
                                 start=1900,
                                 end=2020,
                                 step=1,
                                 value=2004)

halfsize_slider = pn.widgets.IntSlider(name='Half-Size of Time Window',
                                       start=0,
                                       end=50,
                                       step=1,
                                       value=15)
halfsize_slider.format = PrintfTickFormatter(format='%d years')

temp_bias_slider = pn.widgets.FloatSlider(name='Temperatur Bias',
                                          start=-5,
                                          end=5,
                                          step=0.1,
                                          value=0)
temp_bias_slider.format = PrintfTickFormatter(format='%.1f °C')

temp_grad_slider = pn.widgets.FloatSlider(name='Temperatur Gradient',
                                          start=cfg.PARAMS['temp_local_gradient_bounds'][0] * 100,
                                          end=cfg.PARAMS['temp_local_gradient_bounds'][1] * 100,
                                          step=0.01,
                                          value=cfg.PARAMS['temp_default_gradient'] * 100)
temp_grad_slider.format = PrintfTickFormatter(format='%.2f °C/100m')

#busy_indicator_climate_tab = pn.indicators.LoadingSpinner(width=30,
#                                                          height=30,
#                                                          color='light',
#                                                          bgcolor='dark')


def set_climate_button_click(event):
    #toogle_work_status()
    # toogle_busy_indicators()
    # toogle_buttons_dsiabled()
    set_climate_of_mb_model(current_mb_i)
    update_current_figures(current_mb_i)
    # toogle_buttons_dsiabled()
    # toogle_busy_indicators()
    #toogle_work_status()


set_climate_button = pn.widgets.Button(name='Set Climate',
                                       sizing_mode='stretch_width')
set_climate_button.on_click(set_climate_button_click)

climate_tab = pn.Column(base_climate_select,
                        y0_slider,
                        halfsize_slider,
                        temp_bias_slider,
                        temp_grad_slider,
                        set_climate_button,
                        background=menu_background,
                        width=panel_width,
                        sizing_mode='stretch_height')

## MB Settings

In [ ]:
# MB Settings
default_mu = 200
min_mu = 100
max_mu = 300
mu_slider = pn.widgets.IntSlider(name='Sensitivity parameter µ',
                                 start=min_mu,
                                 end=max_mu,
                                 step=10,
                                 value=default_mu)

temp_melt_slider = pn.widgets.FloatSlider(name='T for ice melt (T_melt)',
                                          start=-2,
                                          end=0,
                                          step=0.1,
                                          value=cfg.PARAMS['temp_melt'])

temp_solid_slider = pn.widgets.FloatSlider(name='T for solid Prcp only (T_solid)',
                                           start=-0.5,
                                           end=0.5,
                                           step=0.1,
                                           value=cfg.PARAMS['temp_all_solid'])

temp_liquid_slider = pn.widgets.FloatSlider(name='T for liquid Prcp only (T_liquid)',
                                            start=0.5,
                                            end=3,
                                            step=0.1,
                                            value=cfg.PARAMS['temp_all_liq'])

prcp_fac_slider = pn.widgets.FloatSlider(name='Precipitation Factor',
                                         start=0.5,
                                         end=5,
                                         step=0.1,
                                         value=cfg.PARAMS['prcp_scaling_factor'])


def set_mb_settings_click(event):
    # toogle_work_status()
    # toogle_busy_indicators()
    # toogle_buttons_dsiabled()
    set_mb_settings_of_mb_model(current_mb_i)
    update_current_figures(current_mb_i)
    # toogle_buttons_dsiabled()
    # toogle_busy_indicators()
    # toogle_work_status()


set_mb_settings_button = pn.widgets.Button(name='Set MB Settings',
                                           sizing_mode='stretch_width')
set_mb_settings_button.on_click(set_mb_settings_click)

#busy_indicator_mb_settings_tab = pn.indicators.LoadingSpinner(width=30,
#                                                              height=30,
#                                                              color='light',
#                                                              bgcolor='dark')

MB_settings_tab = pn.Column(mu_slider,
                            temp_melt_slider,
                            temp_solid_slider,
                            temp_liquid_slider,
                            prcp_fac_slider,
                            set_mb_settings_button,
                            background=menu_background,
                            width=panel_width,
                            sizing_mode='stretch_height')

## Toogle busy indicators

In [ ]:
def toogle_busy_indicators():
    busy_indicator_climate_tab.value = not busy_indicator_climate_tab.value
    busy_indicator_mb_settings_tab.value = not busy_indicator_mb_settings_tab.value

## Toogle buttons disabled

In [ ]:
def toogle_buttons_dsiabled():
    set_mb_settings_button.disabled = not set_mb_settings_button.disabled
    set_climate_button.disabled = not set_climate_button.disabled

## But all Tabs together

In [ ]:
tab_menu = pn.Tabs(('Climate Settings', climate_tab),
                   ('MB Settings', MB_settings_tab),
                   height=420,
                   width=tab_menu_width,
                   background=menu_background,
                   tabs_location='left')

## functions to enable and disable menu

In [ ]:
def toogle_disabled_of_menu():
    base_climate_select.disabled = not base_climate_select.disabled
    ref_height_slider.disabled = not ref_height_slider.disabled
    temp_bias_slider.disabled = not temp_bias_slider.disabled
    prcp_fac_slider.disabled = not prcp_fac_slider.disabled
    temp_grad_slider.disabled = not temp_grad_slider.disabled
    prcp_grad_slider.disabled = not prcp_grad_slider.disabled
    set_climate_button.disabled = not set_climate_button.disabled
    mu_slider.disabled = not mu_slider.disabled
    temp_melt_slider.disabled = not temp_melt_slider.disabled
    temp_solid_slider.disabled = not temp_solid_slider.disabled
    temp_liquid_slider.disabled = not temp_liquid_slider.disabled
    set_mb_settings_button.disabled = not set_mb_settings_button.disabled

# Define function for text

In [ ]:
def get_simple_text(text='', width=30, font_size='30pt'):
    return pn.Column(pn.layout.VSpacer(),
                     pn.widgets.StaticText(value=text,
                                           style={'font-size': font_size}),
                     pn.layout.VSpacer(),
                     width=width,
                     sizing_mode="stretch_height",
                     )

In [ ]:
def get_latex_text(text='', width=50, font_size='40pt'):
    return pn.Column(pn.layout.VSpacer(),
                     pn.pane.LaTeX(text,
                                   style={'font-size': font_size}),
                     pn.layout.VSpacer(),
                     width=width,
                     sizing_mode="stretch_height",
                     )

# MassBalanceClass

## Adapted OGGM MassBalanceModels to avoid the need of gdirs

In [ ]:
class PastMassBalanceAdapted(PastMassBalance):
    """Mass balance during the climate data period."""

    def __init__(self, min_height=0, max_height=6000,
                 mu_star=None, bias=None,
                 filename='climate_historical', input_filesuffix='',
                 fpath=climate_data_dir, t_solid=None, t_liq=None,
                 t_melt=None, prcp_fac=None, t_grad=None,
                 repeat=False, ys=None, ye=None):
        """TODO
        """

        super(PastMassBalance, self).__init__()

        self.valid_bounds = [min_height, max_height]  # in m
        self.default_heights = np.linspace(self.valid_bounds[0],
                                           self.valid_bounds[1],
                                           num=50,
                                           endpoint=True)

        if mu_star is None:
            mu_star = default_mu

        if bias is None:
                bias = 0.

        self.mu_star = np.array(mu_star)
        self.bias = np.array(bias)

        # Parameters
        if t_solid is None:
            self.t_solid = np.array(cfg.PARAMS['temp_all_solid'])
        else:
            self.t_solid = np.array(t_solid)
        if t_liq is None:
            self.t_liq = np.array(cfg.PARAMS['temp_all_liq'])
        else:
            self.t_liq = np.array(t_liq)
        if t_melt is None:
            self.t_melt = np.array(cfg.PARAMS['temp_melt'])
        else:
            self.t_melt = np.array(t_melt)
        if prcp_fac is None:
            prcp_fac = np.array(cfg.PARAMS['prcp_scaling_factor'])
        # check if valid prcp_fac is used
        if prcp_fac <= 0:
            raise InvalidParamsError('prcp_fac has to be above zero!')
        if t_grad is None:
            t_grad = cfg.PARAMS['temp_default_gradient']

        # Public attrs
        self.repeat = repeat

        # Private attrs
        # to allow prcp_fac to be changed after instantiation
        # prescribe the prcp_fac as it is instantiated
        self._prcp_fac = prcp_fac
        # same for temp bias
        self._temp_bias = 0.

        # Read file
        with ncDataset(fpath + filename + input_filesuffix + '.nc', mode='r') as nc:
            # time
            time = nc.variables['time']
            try:
                time = netCDF4.num2date(time[:], time.units)
            except ValueError:
                # This is for longer time series
                time = cftime.num2date(time[:], time.units, calendar='noleap')
            ny, r = divmod(len(time), 12)
            if r != 0:
                raise ValueError('Climate data should be N full years')
            # This is where we switch to hydro float year format
            # Last year gives the tone of the hydro year
            self.years = np.repeat(np.arange(time[-1].year-ny+1,
                                             time[-1].year+1), 12)
            self.months = np.tile(np.arange(1, 13), ny)
            # Read timeseries and correct it
            self.temp = nc.variables['temp'][:].astype(np.float64) + self._temp_bias
            self.prcp = nc.variables['prcp'][:].astype(np.float64) * self._prcp_fac
            if 'gradient' in nc.variables:
                grad = nc.variables['gradient'][:].astype(np.float64)
                # Security for stuff that can happen with local gradients
                g_minmax = cfg.PARAMS['temp_local_gradient_bounds']
                grad = np.where(~np.isfinite(grad), t_grad, grad)
                grad = clip_array(grad, g_minmax[0], g_minmax[1])
            else:
                grad = self.prcp * 0 + t_grad
            self.grad = grad
            self.ref_hgt = nc.ref_hgt
            self.ys = self.years[0] if ys is None else ys
            self.ye = self.years[-1] if ye is None else ye

In [ ]:
class ConstantMassBalanceAdapted(ConstantMassBalance):
    """Constant mass-balance during a chosen period.
    This is useful for equilibrium experiments.
    """

    def __init__(self, min_height=0, max_height=6000, mu_star=None, bias=None,
                 y0=None, halfsize=15, filename='climate_historical',
                 input_filesuffix='', **kwargs):
        """TODO
        """

        super(ConstantMassBalance, self).__init__()
        self.mbmod = PastMassBalanceAdapted(min_height=min_height, max_height=max_height,
                                            mu_star=mu_star, bias=bias,
                                            filename=filename,
                                            input_filesuffix=input_filesuffix,
                                            **kwargs)

        if y0 is None:
            y0 = self.mbmod.ye - halfsize

        self.valid_bounds = [min_height, max_height]  # in m
        self.default_heights = np.linspace(self.valid_bounds[0],
                                           self.valid_bounds[1],
                                           num=50,
                                           endpoint=True)
        self.hbins = self.default_heights

        # Private attrs
        # to allow y0 and halfsize to be changed after instantiation
        # to keep years up to date
        self._y0 = y0
        self._halfsize = halfsize
        self.update_years()
        # self.years = np.arange(y0-halfsize, y0+halfsize+1)

    @property
    def y0(self):
        return self._y0

    @y0.setter
    def y0(self, value):
        # check that the new value is in the bounds of the climate data
        if value > self.mbmod.ye - self._halfsize:
            self._y0 = self.mbmod.ye - self._halfsize
        elif value < self.mbmod.ys + self._halfsize:
            self._y0 = self.mbmod.ys + self._halfsize
        else:
            self._y0 = value
        self.update_years()
    
    @property
    def halfsize(self):
        return self._halfsize

    @halfsize.setter
    def halfsize(self, value):
        self._halfsize = value
        self.update_years()
    
    def update_years(self):
        self.years = np.arange(self._y0 - self._halfsize, self._y0 + self._halfsize + 1)
        
    # this two function are coppied of the original ConstantMassBalance Class without
    # the lazy decorator to be able to change y0 and halfsize, and also include 
    # the heights direclty and not returning a function (interp1d)
    def interp_yr(self, heights):
        # annual MB
        mb_on_h = self.hbins*0.
        for yr in self.years:
            mb_on_h += self.mbmod.get_annual_mb(self.hbins, year=yr)
        return interp1d(self.hbins, mb_on_h / len(self.years))(heights)

    def interp_m(self, heights):
        # monthly MB
        months = np.arange(12)+1
        interp_m = []
        for m in months:
            mb_on_h = self.hbins*0.
            for yr in self.years:
                yr = date_to_floatyear(yr, m)
                mb_on_h += self.mbmod.get_monthly_mb(self.hbins, year=yr)
            interp_m.append(interp1d(self.hbins, mb_on_h / len(self.years))(heights))
        return interp_m

## Class for extention of MB Models with visualisations

In [ ]:
class MassBalanceVis(ConstantMassBalanceAdapted):
    """Mass balance during the climate data period."""

    def __init__(self,
                 min_height=0,
                 max_height=6000,
                 input_filesuffix='',
                 mb_model_nr=0,
                 t_grad=None,
                 base_climate_name='',
                 mu=200,
                 y0=None,
                 halfsize=15,
                 t_solid=None,
                 t_liq=None,
                 t_melt=None,
                 prcp_fac=None,
                 repeat=False, ys=None, ye=None):
        """TODO
        """

        super(MassBalanceVis, self).__init__(min_height=min_height,
                                             max_height=max_height,
                                             input_filesuffix=input_filesuffix,
                                             t_grad=t_grad,
                                             mu_star=mu,
                                             y0=y0,
                                             halfsize=halfsize,
                                             t_solid=t_solid,
                                             t_liq=t_liq,
                                             t_melt=t_melt,
                                             prcp_fac=prcp_fac,
                                             repeat=repeat,
                                             ys=ys,
                                             ye=ye
                                             )

        self.base_climate_name = base_climate_name
        self.mb_model_nr = mb_model_nr

        # using months in hydro year
        self.months_short = {'Oct': 1,
                             'Nov': 2,
                             'Dec': 3,
                             'Jan': 4,
                             'Feb': 5,
                             'Mar': 6,
                             'Apr': 7,
                             'May': 8,
                             'Jun': 9,
                             'Jul': 10,
                             'Aug': 11,
                             'Sep': 12}

        self.default_tools = ['xwheel_zoom', 'hover', 'save']
        self.active_tools = ['xwheel_zoom']

    def get_climograph(self, ref_height=None, **kwargs):
        def prcp_hook(plot, element):
            p = plot.state

            # color yaxis of prcp axis
            prcp_color = 'blue'
            plot.handles['yaxis'].axis_label_text_color = prcp_color
            plot.handles['yaxis'].major_label_text_color = prcp_color
            plot.handles['yaxis'].axis_line_color = prcp_color
            plot.handles['yaxis'].major_tick_line_color = prcp_color
            plot.handles['yaxis'].minor_tick_line_color = prcp_color

            # set y_range of first axis
            #glyph = p.renderers[-1]
            #vals = glyph.data_source.data['Precipitation_left_parenthesis_mm_right_parenthesis']
            #plot.handles['y_range'].start = 0
            #plot.handles['y_range'].end = vals.max() * 1.1

        def temp_hook(plot, element):
            p = plot.state

            temp_color = 'red'

            y2_label = 'Temperature (°C)'
            # create secondary range and axis
            if 'twiny' not in [t for t in p.extra_y_ranges]:
                p.y_range = Range1d(start=plot.handles['y_range'].start,
                                    end=plot.handles['y_range'].end)
                p.y_range.name = 'default'
                p.extra_y_ranges = {"twiny": Range1d(start=-10, end=10)}
                p.add_layout(LinearAxis(axis_label=y2_label,
                                        y_range_name="twiny",
                                        axis_label_text_color=temp_color,
                                        major_label_text_color=temp_color,
                                        axis_line_color=temp_color,
                                        major_tick_line_color=temp_color,
                                        minor_tick_line_color=temp_color), 'right')

            # set glyph y_range_name to the one we've just created
            glyph = p.renderers[-1]
            glyph.y_range_name = 'twiny'
            vals = glyph.data_source.data['Temperature']

            # set range of second y axis
            y_add_range = (vals.max() - vals.min()) * 0.05
            p.extra_y_ranges["twiny"].start = vals.min() - y_add_range
            p.extra_y_ranges["twiny"].end = vals.max() + y_add_range

        if ref_height is None:
            ref_height = self.mbmod.ref_hgt

        temp_data = []
        prcp_data = []
        for month_name, month_nr in self.months_short.items():
            temp, tempformelt, prcp_total, prcp_sol = \
                self.get_monthly_climate(heights=[ref_height], year=month_nr/12)
            temp_data = np.append(temp_data, temp)
            prcp_data = np.append(prcp_data, prcp_total / self.mbmod._prcp_fac)

        # this is needed to show Temperatuer and Precipitation in the same hover
        df = pd.DataFrame({'Month': np.array(list(self.months_short.keys())),
                           'MonthLong': np.array(['October',
                                                  'November',
                                                  'December',
                                                  'January',
                                                  'February',
                                                  'March',
                                                  'April',
                                                  'May',
                                                  'June',
                                                  'July',
                                                  'August',
                                                  'September']),
                           'Precipitation': np.array(prcp_data),
                           'Temperature': np.array(temp_data)})
        
        hover = HoverTool(tooltips="""
            <div>
                <span style="font-size: 12px; color: #000000;">Month: @{MonthLong}</span>
                <br />
                <span style="font-size: 12px; color: #0000FF;">Precipitation: @{Precipitation}{%d} mm</span>
                <br />
                <span style="font-size: 12px; color: #FF0000;">Temperature: @{Temperature}{%0.1f} °C</span>
            </div>""",
                          formatters={'@{Precipitation}': 'printf',
                                      '@{Temperature}': 'printf'},
                          mode='vline')
        tools = ['save', hover]
        
        return (hv.Bars(df,
                        kdims='Month',
                        vdims=['Precipitation', 'Temperature', 'MonthLong'],
                        ).opts(color='blue',
                               responsive=True,
                               ylim=(0, max(prcp_data) * 1.1),
                               ylabel='Precipitation (mm)',
                               # min_height=300,
                               hooks=[prcp_hook],
                               xlabel='',
                               default_tools=tools
                               ) *
                hv.Curve(df,
                         vdims='Temperature',
                         kdims='Month'
                         ).opts(color='red',
                                hooks=[temp_hook],
                                responsive=True,
                                # min_height=300,
                                xlabel='',
                                default_tools=['save'] #tools
                                )
                ).opts(title=('Climograph for Period ' + str(self.years[0]) +
                              ' to ' + str(self.years[-1]) + ', at ' + 
                              '{:d} m'.format(int(self.mbmod.ref_hgt))),
                       xrotation=45)

    def get_monthly_components(self, month, **kwargs):
        heights = self.default_heights
        mb, t, tmelt, prcp, prcpsol = self.get_monthly_mb(heights=heights,
                                                          year=self.months_short[month]/12,
                                                          add_climate=True)
        month_acc = prcpsol / self.mbmod.rho
        month_abl = - self.mbmod.mu_star * tmelt / self.mbmod.rho
        return month_acc, month_abl
        
    def get_annual_components(self, **kwargs):
        heights = self.default_heights
        mb, t, tmelt, prcp, prcpsol = self.get_annual_mb(heights=heights, add_climate=True)
        annual_acc = prcpsol / self.mbmod.rho
        annual_abl = - self.mbmod.mu_star * tmelt / self.mbmod.rho
        return annual_acc, annual_abl
        
    def get_xlim_annual(self, **kwargs):
        heights = self.default_heights
        annual_accumulation, annual_ablation = self.get_annual_components()

        return (np.min(annual_ablation) - 5, np.max(annual_accumulation) + 5)

    def get_annual_mb_curve(self, **kwargs):
        heights = self.default_heights
        mb_annual = self.get_annual_mb(heights=heights) * SEC_IN_YEAR

        annual_mb_curve = hv.Curve((mb_annual, heights),
                                   kdims='MB annual' + str(self.mb_model_nr),
                                   vdims='height (m)',
                                   label='annual').opts(responsive=True,
                                                        default_tools=self.default_tools,
                                                        active_tools=self.active_tools,
                                                        xlabel='mm w.e./year',
                                                        ylim=(
                                                            np.min(heights), np.max(heights)),
                                                        xlim=self.get_xlim_annual()
                                                        )

        annual_mb_curve.opts(opts.Curve(framewise=True))

        return annual_mb_curve

    def get_month_mb_curve(self, month=None, **kwargs):
        heights = self.default_heights
        mb_month = self.get_monthly_mb(
            heights=heights, year=self.months_short[month]/12) * SEC_IN_MONTH

        month_mb_curve = hv.Curve((mb_month, heights),
                                  kdims='MB ' + month + str(self.mb_model_nr),
                                  vdims='height (m)',
                                  label=month
                                  ).opts(responsive=True,
                                         default_tools=self.default_tools,
                                         active_tools=self.active_tools,
                                         xlabel='mm w.e./month',
                                         ylim=(np.min(heights),
                                               np.max(heights)),
                                         xlim=(np.min(mb_month) - 1,
                                               np.max(mb_month) + 1)
                                         )

        month_mb_curve.opts(opts.Curve(framewise=True))

        return month_mb_curve

    def get_annual_accumulation_curve(self, **kwargs):
        heights = self.default_heights
        annual_accumulation, annual_ablation = self.get_annual_components()

        annual_accumulation_curve = hv.Curve((annual_accumulation, heights),
                                             kdims='Acc. annual' +
                                             str(self.mb_model_nr),
                                             vdims='height (m)',
                                             label='annual'
                                             ).opts(responsive=True,
                                                    default_tools=self.default_tools,
                                                    active_tools=self.active_tools,
                                                    xlabel='mm w.e./year',
                                                    ylim=(np.min(heights),
                                                          np.max(heights)),
                                                    xlim=self.get_xlim_annual()
                                                    )

        annual_accumulation_curve.opts(opts.Curve(framewise=True))
        return annual_accumulation_curve

    def get_annual_ablation_curve(self, **kwargs):
        heights = self.default_heights
        annual_accumulation, annual_ablation = self.get_annual_components()

        annual_ablation_curve = hv.Curve((annual_ablation, heights),
                                         kdims='Abl. annual' +
                                         str(self.mb_model_nr),
                                         vdims='height (m)',
                                         label='annual'
                                         ).opts(responsive=True,
                                                default_tools=self.default_tools,
                                                active_tools=self.active_tools,
                                                xlabel='mm w.e./year',
                                                ylim=(np.min(heights),
                                                      np.max(heights)),
                                                xlim=self.get_xlim_annual()
                                                )

        annual_ablation_curve.opts(opts.Curve(framewise=True))

        return annual_ablation_curve

    def get_month_accumulation_curve(self, month=None, **kwargs):
        heights = self.default_heights
        month_accumulation, month_ablation = self.get_monthly_components(heights=heights, month=month)

        month_accumulation_curve = hv.Curve((month_accumulation, heights),
                                            kdims='Acc. ' + month +
                                            str(self.mb_model_nr),
                                            vdims='height (m)',
                                            label=month
                                            ).opts(responsive=True,
                                                   default_tools=self.default_tools,
                                                   active_tools=self.active_tools,
                                                   xlabel='mm w.e./month',
                                                   ylim=(np.min(heights),
                                                         np.max(heights)),
                                                   xlim=self.get_xlim_annual()
                                                   )

        month_accumulation_curve.opts(opts.Curve(framewise=True))

        return month_accumulation_curve

    def get_month_ablation_curve(self, month=None, **kwarg):
        heights = self.default_heights
        month_accumulation, month_ablation = self.get_monthly_components(heights=heights, month=month)

        month_ablation_curve = hv.Curve((month_ablation, heights),
                                        kdims='Abl. ' + month +
                                        str(self.mb_model_nr),
                                        vdims='height (m)',
                                        label=month
                                        ).opts(responsive=True,
                                               default_tools=self.default_tools,
                                               active_tools=self.active_tools,
                                               xlabel='mm w.e./month',
                                               ylim=(np.min(heights),
                                                     np.max(heights)),
                                               xlim=self.get_xlim_annual()
                                               )

        month_ablation_curve.opts(opts.Curve(framewise=True))

        return month_ablation_curve

    def get_ELA_curve(self, kdims=None, **kwarg):
        heights = self.default_heights

        ELA = self.get_ela()

        ELA_curve = hv.Curve((list(self.get_xlim_annual()),
                              [ELA, ELA]),
                             kdims=kdims + str(self.mb_model_nr),
                             vdims='height (m)',
                             label='ELA'
                             ).opts(responsive=True,
                                    color='black',
                                    line_dash='dashed',
                                    default_tools=self.default_tools,
                                    active_tools=self.active_tools,
                                    ylim=(np.min(heights), np.max(heights)),
                                    xlim=self.get_xlim_annual()
                                    )

        ELA_curve.opts(opts.Curve(framewise=True))

        return ELA_curve

# Define default MassBalanceModels

In [ ]:
mb_models = [MassBalanceVis(
                    min_height=min_height,
                    max_height=max_height,
                    input_filesuffix=base_climate_names['Hintereisferner (Mid Latitudes)'],
                    mb_model_nr=0,
                    t_grad=temp_grad_slider.value / 100,
                    base_climate_name='Hintereisferner (Mid Latitudes)',
                    mu=mu_slider.value,
                    y0=y0_slider.value,
                    halfsize=halfsize_slider.value,
                    t_solid=temp_solid_slider.value,
                    t_liq=temp_liquid_slider.value,
                    t_melt=temp_melt_slider.value,
                    prcp_fac=prcp_fac_slider.value),
             MassBalanceVis(
                    min_height=min_height,
                    max_height=max_height,
                    input_filesuffix=base_climate_names['Lewis (Tropics)'],
                    mb_model_nr=1,
                    t_grad=temp_grad_slider.value / 100,
                    base_climate_name='Lewis (Tropics)',
                    mu=mu_slider.value,
                    y0=y0_slider.value,
                    halfsize=halfsize_slider.value,
                    t_solid=temp_solid_slider.value,
                    t_liq=temp_liquid_slider.value,
                    t_melt=temp_melt_slider.value,
                    prcp_fac=prcp_fac_slider.value)]

# Connect MB Models with menu

## climate

In [ ]:
def set_climate_of_mb_model(mb_i):
    global mb_models

    # save the old MB settings to let them unchanged
    mu_old = mb_models[mb_i].mbmod.mu_star
    t_melt_old = mb_models[mb_i].mbmod.t_melt
    t_solid_old = mb_models[mb_i].mbmod.t_solid
    t_liq_old = mb_models[mb_i].mbmod.t_liq
    prcp_fac_old = mb_models[mb_i].prcp_fac

    # initialise new MB with new climate settings
    mb_models[mb_i] = MassBalanceVis(
        min_height=min_height,
        max_height=max_height,
        input_filesuffix=base_climate_names[base_climate_select.value],
        mb_model_nr=mb_i,
        t_grad=temp_grad_slider.value / 100,
        base_climate_name=base_climate_select.value,
        mu=mu_old,
        y0=y0_slider.value,
        halfsize=halfsize_slider.value,
        t_solid=t_solid_old,
        t_liq=t_liq_old,
        t_melt=t_melt_old,
        prcp_fac=prcp_fac_old)
    mb_models[mb_i].temp_bias = temp_bias_slider.value

In [ ]:
def set_climate_menu_with_selected_mb_model(mb_i):
    base_climate_select.value = mb_models[mb_i].base_climate_name
    ref_height_slider.value = int(mb_models[mb_i].mbmod.ref_hgt)
    temp_bias_slider.value = float(mb_models[mb_i].mbmod.temp_bias)
    temp_grad_slider.value = float(mb_models[mb_i].mbmod.grad) * 100

## MB settings

In [ ]:
def set_mb_settings_of_mb_model(mb_i):
    mb_models[mb_i].mbmod.mu_star = np.array(mu_slider.value)
    mb_models[mb_i].mbmod.t_melt = np.array(temp_melt_slider.value)
    mb_models[mb_i].mbmod.t_solid = np.array(temp_solid_slider.value)
    mb_models[mb_i].mbmod.t_liq = np.array(temp_liquid_slider.value)
    mb_models[mb_i].prcp_fac = np.array(prcp_fac_slider.value)

In [ ]:
def set_mb_settings_menu_with_selected_mb_model(mb_i):
    mu_slider.value = int(mb_models[mb_i].mbmod.mu_star)
    temp_melt_slider.value = float(mb_models[mb_i].mbmod.t_melt)
    temp_solid_slider.value = float(mb_models[mb_i].mbmod.t_solid)
    temp_liquid_slider.value = float(mb_models[mb_i].mbmod.t_liq)
    prcp_fac_slider.value = float(mb_models[mb_i].mbmod.prcp_fac)

# Coarse plots

In [ ]:
def dyn_overlay_accumulation(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='Acc.')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_accumulation_curve()
    for month in list(mb_models[mb_i].months_short.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_accumulation_curve(
                month=month)
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Accumulation',
                        toolbar=None,
                        show_legend=True,
                        legend_position='top_left',
                        )


def dyn_overlay_ablation(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='Abl.')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_ablation_curve()
    for month in list(mb_models[mb_i].months_short.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_ablation_curve(month=month)
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Ablation',
                        ylabel='',
                        toolbar=None,
                        show_legend=False,
                        )


def dyn_overlay_total_mb(months, update, mb_i=0):
    overlay = mb_models[mb_i].get_ELA_curve(kdims='MB')
    overlay *= hv.VLine(0).opts(color='gray',
                                line_width=1,
                                line_alpha=0.5,
                                default_tools=mb_models[mb_i].default_tools,
                                active_tools=mb_models[mb_i].active_tools,
                                )
    overlay *= mb_models[mb_i].get_annual_mb_curve()
    for month in list(mb_models[mb_i].months_short.keys()):
        if month in months:
            overlay *= mb_models[mb_i].get_month_mb_curve(
                month=month).opts(xlim=(None, None))
        else:
            overlay *= hv.Curve((0, 0), label='').opts(responsive=True,
                                                       default_tools=mb_models[mb_i].default_tools,
                                                       active_tools=mb_models[mb_i].active_tools,
                                                       xlabel='mm w.e./month',
                                                       ylim=(np.min(mb_models[mb_i].default_heights),
                                                             np.max(mb_models[mb_i].default_heights))
                                                       )
    return overlay.opts(title='Total Mass-Balance',
                        ylabel='',
                        toolbar=None,
                        show_legend=False,
                        )


select_months_coarse_0 = pn.widgets.CheckBoxGroup(
    name='month selection',
    value=[],
    options=list(mb_models[0].months_short.keys()),
    inline=False,
    width=50)
select_months_coarse_1 = pn.widgets.CheckBoxGroup(
    name='month selection',
    value=[],
    options=list(mb_models[1].months_short.keys()),
    inline=False,
    width=50)
select_months_coarse_both = [select_months_coarse_0,
                             select_months_coarse_1]

coarse_updater_0 = pn.widgets.Checkbox(name='Update Coarse 0')
coarse_updater_1 = pn.widgets.Checkbox(name='Update Coarse 1')
coarse_updater_both = [coarse_updater_0,
                         coarse_updater_1]

dmap_accumulation_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_accumulation, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_accumulation, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]

dmap_ablation_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_ablation, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_ablation, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]

dmap_total_mb_both = [hv.DynamicMap(
    pn.bind(partial(dyn_overlay_total_mb, mb_i=0),
            months=select_months_coarse_0,
            update=coarse_updater_0)),
    hv.DynamicMap(
    pn.bind(partial(dyn_overlay_total_mb, mb_i=1),
            months=select_months_coarse_1,
            update=coarse_updater_1)), ]


def get_complete_coarse_figure(mb_i):
    return pn.Row(pn.Column(pn.layout.VSpacer(),
                            select_months_coarse_both[mb_i],
                            pn.layout.VSpacer(),
                            width=50,
                            sizing_mode="stretch_height",
                            ),
                  pn.Row(dmap_accumulation_both[mb_i],
                         get_simple_text('+'),
                         dmap_ablation_both[mb_i],
                         get_simple_text('='),
                         dmap_total_mb_both[mb_i],
                         sizing_mode='stretch_both'
                         ),
                  sizing_mode='stretch_both',
                  )


coarse_figures = [get_complete_coarse_figure(0),
                  get_complete_coarse_figure(1)]

In [ ]:
def toogle_coarse_disabled():
    select_months_coarse_0.disabled = not select_months_coarse_0.disabled
    select_months_coarse_1.disabled = not select_months_coarse_1.disabled

# Climographs

In [ ]:
def get_climograph(change, mb_i=0):
    return mb_models[mb_i].get_climograph().opts(toolbar=None,
                                                 height=250)

climograph_updater_0 = pn.widgets.Checkbox(name='Update Climograph 0')
climograph_updater_1 = pn.widgets.Checkbox(name='Update Climograph 1')
climograph_updater_both = [climograph_updater_0,
                           climograph_updater_1]

climographs = [hv.DynamicMap(
    pn.bind(partial(get_climograph, mb_i=0),
            change=climograph_updater_0)),
               hv.DynamicMap(
    pn.bind(partial(get_climograph, mb_i=1),
            change=climograph_updater_1)),]

# Table for current settings

In [ ]:
def get_table(change, mb_i=0):
    variables = ['T_bias', 'T_grad', 'Prcp_fac', 'µ', 'T_melt', 'T_solid', 'T_liquid']
    units = ['°C', '°C/100m', ' ', ' ' , '°C', '°C', '°C']
    values = [mb_models[mb_i].temp_bias,
              mb_models[mb_i].mbmod.grad[0] * 100,
              mb_models[mb_i].mbmod.prcp_fac,
              mb_models[mb_i].mbmod.mu_star,
              mb_models[mb_i].mbmod.t_melt,
              mb_models[mb_i].mbmod.t_solid,
              mb_models[mb_i].mbmod.t_liq]

    return hv.Table({'Variable': variables, 'Value': values, 'Unit':units},
             ['Variable'], ['Value', 'Unit'],
             ).opts(index_position=None,
                    width=160,
                    height=210
                   )

info_table_updater_0 = pn.widgets.Checkbox(name='Update Info table 0')
info_table_updater_1 = pn.widgets.Checkbox(name='Update Info table 1')
info_table_updater_both = [info_table_updater_0,
                           info_table_updater_1]

info_tables = [hv.DynamicMap(
    pn.bind(partial(get_table, mb_i=0),
            change=info_table_updater_0)),
               hv.DynamicMap(
    pn.bind(partial(get_table, mb_i=1),
            change=info_table_updater_1))]

# Function to toogle all things during work

In [ ]:
def toogle_work_status():
    toogle_busy_indicators()
    toogle_disabled_of_menu()
    toogle_detailed_disabled()
    toogle_coarse_disabled()

# Function to update figures

In [ ]:
def update_current_figures(mb_i):
    climograph_updater_both[mb_i].value = not climograph_updater_both[mb_i].value
    coarse_updater_both[mb_i].value = not coarse_updater_both[mb_i].value
    info_table_updater_both[mb_i].value = not info_table_updater_both[mb_i].value

# Put figures together

## help function

In [ ]:
def get_mb_tab(i):
    global mb_models

    # grid = pn.GridSpec(sizing_mode='stretch_both',
    #                   mode='override')
    #grid[0, 0] = climographs[i]
    # grid[0, 1:3] = pn.Column(current_mb_settings_text,
    #                         pn.Card(mb_equation,
    #                                 pn.layout.Divider(),
    #                                 f_equation,
    #                                 title='Mass Balance Model Equations',
    #                                 collapsed=True),
    #                         background='whitesmoke',
    #                         sizing_mode='stretch_height')
    #grid[1, :] = coarse_figures[i]
    return pn.Column(pn.Row(climographs[i],
                            info_tables[i],
                            sizing_mode='stretch_width'),
                     pn.Row(coarse_figures[i],
                            sizing_mode='stretch_width'),
                     sizing_mode='stretch_both')

## create figures tab

In [ ]:
def change_figures_tab(event):
    global current_mb_i

    if event.new == 0:
        # if base_climate_select.disabled:
        #    toogle_disabled_of_menu()
        current_mb_i = 0
        set_climate_menu_with_selected_mb_model(0)
        set_mb_settings_menu_with_selected_mb_model(0)
        update_info_text(0)

    elif event.new == 1:
        # if base_climate_select.disabled:
        #    toogle_disabled_of_menu()
        current_mb_i = 1
        set_climate_menu_with_selected_mb_model(1)
        set_mb_settings_menu_with_selected_mb_model(1)
        update_info_text(1)

    elif event.new == 2:
        pass
        # if not base_climate_select.disabled:
        #    toogle_disabled_of_menu()


figures_tab = pn.Tabs(('Mass Balance Model 1', get_mb_tab(0)),
                      ('Mass Balance Model 2', get_mb_tab(1)),
                      ('Compare Mass Balance Models', pn.Column(
                          pn.pane.Markdown(object=('<p style="margin-top: 0px;">' +
                                                   'TODO...'
                                                   + '</p>'),
                                           # height=20,
                                           margin=(0, 0),
                                           align='end',),
                                                              )
                      ),
                     )
figures_tab.param.watch(change_figures_tab, ['active']);

# Put App together

In [ ]:
initial_run = False

In [ ]:
app = pn.Row(tab_menu,
             figures_tab,
             sizing_mode='stretch_both')

In [ ]:
app.servable(title='Mass Balance Simulator')